In [1]:
import gym

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import random

import math

In [2]:
class CNet(nn.Module):
    def __init__(self, in_, out_, hidden=128):
        super(CNet, self).__init__()
        self.mu = nn.Sequential(
            nn.Linear(in_,hidden),
            nn.ReLU(),
            nn.Linear(hidden,out_),
            nn.Tanh()
        )
        self.var = nn.Sequential(
            nn.Linear(in_,hidden),
            nn.ReLU(),
            nn.Linear(hidden,out_),
            nn.Softplus()
        )
        self.value = nn.Sequential(
            nn.Linear(in_,hidden),
            nn.ReLU(),
            nn.Linear(hidden,1)
        )
    
    def forward(self, x):
        return self.mu(x), self.var(x), self.value(x)

In [7]:
env = gym.make("Pendulum-v0")

GAMMA = 0.999
LR = 0.001
EPOCH = 50000

ENT_SCALE = 0.01

BATCH = 10
VIDEO = 400000

in_ = env.observation_space.shape[0]
out_ = env.action_space.shape[0]
high_ = env.action_space.high[0]
low_ = env.action_space.low[0]

In [4]:
net = CNet(in_,out_)
opt = optim.Adam(net.parameters(), lr=LR)

In [8]:
batch_cnt = 0
batch_obs = []
batch_act = []
batch_qval = []
for epoch in range(EPOCH):
    obs = env.reset()
    if epoch%VIDEO == 0:
        env.render()
    cur_rew = []
    count = 0
    while True:
        with torch.no_grad():
            mu, var, _ = net(torch.FloatTensor(obs))
            mu = mu.data.numpy()
            sig = torch.sqrt(var).data.numpy()
            act = np.random.normal(mu, sig)
            act = np.clip(act, low_, high_)
        
        next_obs, rew, done, _ = env.step(act)
        if epoch%VIDEO == 0:
            env.render()
        count+=1
        
        batch_obs.append(obs)
        batch_act.append(act)
        cur_rew.append(rew)
        
        obs = next_obs

        if done:
            qval = []
            r_sum = 0
            for r in reversed(cur_rew):
                r_sum*= GAMMA
                r_sum+= r
                qval.append(r_sum)
            batch_qval.extend(list(reversed(qval)))
            batch_cnt+=1
            break
    print("epoch : %d count : %d rew : %d"%(epoch, count, rew))
    
    if batch_cnt == BATCH:
        obss = torch.FloatTensor(batch_obs)
        acts = torch.FloatTensor(batch_act)
        qvals = torch.FloatTensor(batch_qval).unsqueeze(1)

        opt.zero_grad()

        # use network to get logit, value
        mu, var, value = net(obss)

        # value -> need to be similar to qvals.
        value_loss = F.mse_loss(value, qvals)

        # policy gradient. using advatange score to reduce variance.
        first = -((mu - acts)**2/2*var.clamp(1e-3))
        two = - torch.log(torch.sqrt(2*math.pi*var))
        
        log_prob = first + two
        adv = qvals - value.detach()
        policy_value = adv * log_prob
        policy_loss = -policy_value.mean()

        # using entropy to make agent explore more.
        entropy_loss = ENT_SCALE * (-(torch.log(2*math.pi*var)+1)/2).mean()

        loss = value_loss + policy_loss + entropy_loss
        print(value_loss, policy_loss, entropy_loss)
        loss.backward()

        opt.step()

        batch_cnt = 0
        batch_obs = []
        batch_act = []
        batch_qval = []
env.close()

epoch : 0 count : 200 rew : -2
epoch : 1 count : 200 rew : -3
epoch : 2 count : 200 rew : -2
epoch : 3 count : 200 rew : -8
epoch : 4 count : 200 rew : -9
epoch : 5 count : 200 rew : -7
epoch : 6 count : 200 rew : -7
epoch : 7 count : 200 rew : -4
epoch : 8 count : 200 rew : -16
epoch : 9 count : 200 rew : -4
tensor(216038.0781, grad_fn=<MseLossBackward>) tensor(-117887.2422, grad_fn=<NegBackward>) tensor(-0.0443, grad_fn=<MulBackward0>)
epoch : 10 count : 200 rew : -6
epoch : 11 count : 200 rew : -10
epoch : 12 count : 200 rew : -15
epoch : 13 count : 200 rew : -9
epoch : 14 count : 200 rew : -10
epoch : 15 count : 200 rew : -8
epoch : 16 count : 200 rew : -11
epoch : 17 count : 200 rew : -5
epoch : 18 count : 200 rew : -9
epoch : 19 count : 200 rew : -7
tensor(220183.8281, grad_fn=<MseLossBackward>) tensor(-187336.3125, grad_fn=<NegBackward>) tensor(-0.0444, grad_fn=<MulBackward0>)
epoch : 20 count : 200 rew : -5
epoch : 21 count : 200 rew : -8
epoch : 22 count : 200 rew : -8
epoch :

epoch : 180 count : 200 rew : -1
epoch : 181 count : 200 rew : -2
epoch : 182 count : 200 rew : -8
epoch : 183 count : 200 rew : -5
epoch : 184 count : 200 rew : -3
epoch : 185 count : 200 rew : -3
epoch : 186 count : 200 rew : -4
epoch : 187 count : 200 rew : -7
epoch : 188 count : 200 rew : -4
epoch : 189 count : 200 rew : -8
tensor(213880.9688, grad_fn=<MseLossBackward>) tensor(-133770.4062, grad_fn=<NegBackward>) tensor(-0.0444, grad_fn=<MulBackward0>)
epoch : 190 count : 200 rew : -5
epoch : 191 count : 200 rew : -6
epoch : 192 count : 200 rew : -10
epoch : 193 count : 200 rew : -1
epoch : 194 count : 200 rew : -4
epoch : 195 count : 200 rew : -4
epoch : 196 count : 200 rew : -7
epoch : 197 count : 200 rew : -6
epoch : 198 count : 200 rew : -1
epoch : 199 count : 200 rew : -3
tensor(177789.8281, grad_fn=<MseLossBackward>) tensor(-18285.2070, grad_fn=<NegBackward>) tensor(-0.0441, grad_fn=<MulBackward0>)
epoch : 200 count : 200 rew : -3
epoch : 201 count : 200 rew : -8
epoch : 202 

tensor(171553.0781, grad_fn=<MseLossBackward>) tensor(-61936.8516, grad_fn=<NegBackward>) tensor(-0.0444, grad_fn=<MulBackward0>)
epoch : 360 count : 200 rew : 0
epoch : 361 count : 200 rew : -3
epoch : 362 count : 200 rew : -7
epoch : 363 count : 200 rew : -1
epoch : 364 count : 200 rew : -3
epoch : 365 count : 200 rew : -12
epoch : 366 count : 200 rew : -6
epoch : 367 count : 200 rew : -5
epoch : 368 count : 200 rew : -5
epoch : 369 count : 200 rew : -2
tensor(174295.3438, grad_fn=<MseLossBackward>) tensor(1857.2720, grad_fn=<NegBackward>) tensor(-0.0443, grad_fn=<MulBackward0>)
epoch : 370 count : 200 rew : -2
epoch : 371 count : 200 rew : -2
epoch : 372 count : 200 rew : -1
epoch : 373 count : 200 rew : 0
epoch : 374 count : 200 rew : -5
epoch : 375 count : 200 rew : -6
epoch : 376 count : 200 rew : -2
epoch : 377 count : 200 rew : -14
epoch : 378 count : 200 rew : -7
epoch : 379 count : 200 rew : -5
tensor(171511.9688, grad_fn=<MseLossBackward>) tensor(15388.8604, grad_fn=<NegBack

epoch : 536 count : 200 rew : -11
epoch : 537 count : 200 rew : -6
epoch : 538 count : 200 rew : -4
epoch : 539 count : 200 rew : -3
tensor(182178.6562, grad_fn=<MseLossBackward>) tensor(-127345.2109, grad_fn=<NegBackward>) tensor(-0.0446, grad_fn=<MulBackward0>)
epoch : 540 count : 200 rew : -7
epoch : 541 count : 200 rew : -7
epoch : 542 count : 200 rew : 0
epoch : 543 count : 200 rew : -5
epoch : 544 count : 200 rew : -14
epoch : 545 count : 200 rew : -5
epoch : 546 count : 200 rew : -2
epoch : 547 count : 200 rew : -9
epoch : 548 count : 200 rew : -4
epoch : 549 count : 200 rew : -4
tensor(151322.3750, grad_fn=<MseLossBackward>) tensor(40103.5625, grad_fn=<NegBackward>) tensor(-0.0445, grad_fn=<MulBackward0>)
epoch : 550 count : 200 rew : -9
epoch : 551 count : 200 rew : -10
epoch : 552 count : 200 rew : -3
epoch : 553 count : 200 rew : 0
epoch : 554 count : 200 rew : -8
epoch : 555 count : 200 rew : -12
epoch : 556 count : 200 rew : -9
epoch : 557 count : 200 rew : 0
epoch : 558 c

epoch : 712 count : 200 rew : -9
epoch : 713 count : 200 rew : -9
epoch : 714 count : 200 rew : -8
epoch : 715 count : 200 rew : -8
epoch : 716 count : 200 rew : -6
epoch : 717 count : 200 rew : -5
epoch : 718 count : 200 rew : -11
epoch : 719 count : 200 rew : -2
tensor(238685.0938, grad_fn=<MseLossBackward>) tensor(-171789.4531, grad_fn=<NegBackward>) tensor(-0.0443, grad_fn=<MulBackward0>)
epoch : 720 count : 200 rew : -6
epoch : 721 count : 200 rew : -3
epoch : 722 count : 200 rew : -6
epoch : 723 count : 200 rew : -6
epoch : 724 count : 200 rew : -9
epoch : 725 count : 200 rew : -5
epoch : 726 count : 200 rew : -3
epoch : 727 count : 200 rew : -6
epoch : 728 count : 200 rew : -1
epoch : 729 count : 200 rew : -10
tensor(188334.9062, grad_fn=<MseLossBackward>) tensor(-42857.9297, grad_fn=<NegBackward>) tensor(-0.0445, grad_fn=<MulBackward0>)
epoch : 730 count : 200 rew : -11
epoch : 731 count : 200 rew : -8
epoch : 732 count : 200 rew : -11
epoch : 733 count : 200 rew : -4
epoch : 7

...

tensor(121162.6797, grad_fn=<MseLossBackward>) tensor(-44370.2109, grad_fn=<NegBackward>) tensor(-0.0471, grad_fn=<MulBackward0>)
epoch : 49180 count : 200 rew : -2
epoch : 49181 count : 200 rew : -5
epoch : 49182 count : 200 rew : -11
epoch : 49183 count : 200 rew : -13
epoch : 49184 count : 200 rew : -2
epoch : 49185 count : 200 rew : -7
epoch : 49186 count : 200 rew : -6
epoch : 49187 count : 200 rew : -4
epoch : 49188 count : 200 rew : -3
epoch : 49189 count : 200 rew : -1
tensor(108098.9297, grad_fn=<MseLossBackward>) tensor(-5344.8857, grad_fn=<NegBackward>) tensor(-0.0471, grad_fn=<MulBackward0>)
epoch : 49190 count : 200 rew : -6
epoch : 49191 count : 200 rew : -8
epoch : 49192 count : 200 rew : 0
epoch : 49193 count : 200 rew : -9
epoch : 49194 count : 200 rew : -7
epoch : 49195 count : 200 rew : -4
epoch : 49196 count : 200 rew : -9
epoch : 49197 count : 200 rew : -6
epoch : 49198 count : 200 rew : -7
epoch : 49199 count : 200 rew : -6
tensor(132876.7812, grad_fn=<MseLossBack

tensor(119979.1953, grad_fn=<MseLossBackward>) tensor(21681.9355, grad_fn=<NegBackward>) tensor(-0.0473, grad_fn=<MulBackward0>)
epoch : 49350 count : 200 rew : -15
epoch : 49351 count : 200 rew : -2
epoch : 49352 count : 200 rew : -4
epoch : 49353 count : 200 rew : -3
epoch : 49354 count : 200 rew : -8
epoch : 49355 count : 200 rew : -3
epoch : 49356 count : 200 rew : -4
epoch : 49357 count : 200 rew : -3
epoch : 49358 count : 200 rew : -4
epoch : 49359 count : 200 rew : -4
tensor(115159.7891, grad_fn=<MseLossBackward>) tensor(-17256.3789, grad_fn=<NegBackward>) tensor(-0.0469, grad_fn=<MulBackward0>)
epoch : 49360 count : 200 rew : -3
epoch : 49361 count : 200 rew : -2
epoch : 49362 count : 200 rew : -8
epoch : 49363 count : 200 rew : -1
epoch : 49364 count : 200 rew : -12
epoch : 49365 count : 200 rew : -7
epoch : 49366 count : 200 rew : -4
epoch : 49367 count : 200 rew : -7
epoch : 49368 count : 200 rew : -9
epoch : 49369 count : 200 rew : -2
tensor(123452.4141, grad_fn=<MseLossBac

tensor(129266.7734, grad_fn=<MseLossBackward>) tensor(-58834.2656, grad_fn=<NegBackward>) tensor(-0.0475, grad_fn=<MulBackward0>)
epoch : 49520 count : 200 rew : -2
epoch : 49521 count : 200 rew : -7
epoch : 49522 count : 200 rew : -5
epoch : 49523 count : 200 rew : -9
epoch : 49524 count : 200 rew : -6
epoch : 49525 count : 200 rew : -6
epoch : 49526 count : 200 rew : -2
epoch : 49527 count : 200 rew : -1
epoch : 49528 count : 200 rew : -9
epoch : 49529 count : 200 rew : -12
tensor(111712.3516, grad_fn=<MseLossBackward>) tensor(-7364.8638, grad_fn=<NegBackward>) tensor(-0.0472, grad_fn=<MulBackward0>)
epoch : 49530 count : 200 rew : -6
epoch : 49531 count : 200 rew : 0
epoch : 49532 count : 200 rew : -3
epoch : 49533 count : 200 rew : -4
epoch : 49534 count : 200 rew : -9
epoch : 49535 count : 200 rew : -9
epoch : 49536 count : 200 rew : -7
epoch : 49537 count : 200 rew : -11
epoch : 49538 count : 200 rew : -3
epoch : 49539 count : 200 rew : 0
tensor(103177.0078, grad_fn=<MseLossBackw

tensor(111984.9766, grad_fn=<MseLossBackward>) tensor(17915.3105, grad_fn=<NegBackward>) tensor(-0.0475, grad_fn=<MulBackward0>)
epoch : 49690 count : 200 rew : -4
epoch : 49691 count : 200 rew : -9
epoch : 49692 count : 200 rew : -1
epoch : 49693 count : 200 rew : -9
epoch : 49694 count : 200 rew : -2
epoch : 49695 count : 200 rew : -5
epoch : 49696 count : 200 rew : -5
epoch : 49697 count : 200 rew : -14
epoch : 49698 count : 200 rew : -6
epoch : 49699 count : 200 rew : -3
tensor(93735.5156, grad_fn=<MseLossBackward>) tensor(51982.9570, grad_fn=<NegBackward>) tensor(-0.0466, grad_fn=<MulBackward0>)
epoch : 49700 count : 200 rew : 0
epoch : 49701 count : 200 rew : -8
epoch : 49702 count : 200 rew : -5
epoch : 49703 count : 200 rew : -9
epoch : 49704 count : 200 rew : -2
epoch : 49705 count : 200 rew : -5
epoch : 49706 count : 200 rew : -1
epoch : 49707 count : 200 rew : -3
epoch : 49708 count : 200 rew : -2
epoch : 49709 count : 200 rew : -2
tensor(111229.9766, grad_fn=<MseLossBackwar

tensor(107311.2656, grad_fn=<MseLossBackward>) tensor(9886.3633, grad_fn=<NegBackward>) tensor(-0.0467, grad_fn=<MulBackward0>)
epoch : 49860 count : 200 rew : -12
epoch : 49861 count : 200 rew : -8
epoch : 49862 count : 200 rew : -9
epoch : 49863 count : 200 rew : -4
epoch : 49864 count : 200 rew : 0
epoch : 49865 count : 200 rew : -14
epoch : 49866 count : 200 rew : -8
epoch : 49867 count : 200 rew : -6
epoch : 49868 count : 200 rew : -6
epoch : 49869 count : 200 rew : -7
tensor(128932.4062, grad_fn=<MseLossBackward>) tensor(-76165.9375, grad_fn=<NegBackward>) tensor(-0.0471, grad_fn=<MulBackward0>)
epoch : 49870 count : 200 rew : -1
epoch : 49871 count : 200 rew : -1
epoch : 49872 count : 200 rew : -10
epoch : 49873 count : 200 rew : -3
epoch : 49874 count : 200 rew : -7
epoch : 49875 count : 200 rew : -8
epoch : 49876 count : 200 rew : -2
epoch : 49877 count : 200 rew : -9
epoch : 49878 count : 200 rew : 0
epoch : 49879 count : 200 rew : -9
tensor(115371.0078, grad_fn=<MseLossBackw